# Туториал по использованию предобученной модели RuBERT в задаче классификации твитов по тональности

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers

In [3]:
# загружаем 5000 позитивных и негативных твитов

df_tweets = pd.read_csv('train_data.csv', sep='\t')
df_tweets = df_tweets.dropna()

In [19]:
fin_data = pd.read_csv('fin_data2.csv', sep='\t')

In [4]:
df_tweets.head(2)

,sentence,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel,label
0,"Джеймс «Бадди» Макгирт (James (Buddy) McGirt, ...",спортсмена,PROFESSION,86,96,0
1,«За всю нашу долгую карьеру нам довелось играт...,музыкантов,PROFESSION,258,268,0


In [20]:
df_fin = fin_data.drop(['entity', 'entity_tag', 'entity_pos_start_rel', 'entity_pos_end_rel'], axis=1)

In [6]:
# df_tweets = df_tweets.drop(df_tweets.tail(37).index)

In [7]:
df_tweets.shape

(6600, 2)

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("AlonCohen/RuSentNE-test")

model_rubert = AutoModelForSequenceClassification.from_pretrained("AlonCohen/RuSentNE-test")

In [21]:
# токенизируем текст каждого твита, для BERT не требуется никакая дополнительная предобработка, лемматизация и прочее

tokenized = fin_data['sentence'].astype(str).str[:512].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)) )

In [10]:
df_tweets['sentence'].astype(str).str[:512]

0       Джеймс «Бадди» Макгирт (James (Buddy) McGirt, ...
1       «За всю нашу долгую карьеру нам довелось играт...
2       Ранее, 7 декабря, толпа болельщиков перекрыла ...
3       В субботу, 21 июля 2018 года, на арене СК «Оли...
4       Представитель талибов Забиулла Муджахид в твит...
                              ...                        
6595    В заявлении британского правительства говоритс...
6596    Накануне вечером около 150 тыс. человек вышли ...
6597    Высший руководитель Ирана, великий аятолла Али...
6598    В заявку сборной России под руководством Влади...
6599    Он был арестован на территории Великобритании ...
Name: sentence, Length: 6600, dtype: object

In [22]:
# Пример токенизации текста: на входе - текст, а на выходе имеем массив с номерами токенов из словаря модели BERT

print(fin_data['sentence'][0])
print(tokenized[0])
print(tokenizer.tokenize(fin_data['sentence'][0]))

Абдул Реза Шайхулислами выразил готовность Ирана оказать содействие правительству Афганистана в учреждении курсов профессионального обучения для безработных афганцев.
[1, 31899, 28454, 65150, 25729, 97693, 59085, 96901, 63239, 26678, 111610, 89144, 29652, 39759, 36882, 111948, 68337, 507, 115840, 78799, 44374, 81506, 6009, 12792, 76876, 49429, 24832, 31344, 38937, 78490, 56607, 507, 60517, 77689, 113516, 31899, 37302, 59812, 33824, 115867, 6009, 11283, 507, 69819, 34851, 66767, 120855, 115881, 25246, 33165, 37302, 44979, 58491, 62167, 80300, 42642, 42120, 69819, 43219, 33189, 87057, 77667, 33653, 11098, 37302, 19351, 39125, 77235, 20128, 260, 2]
['▁А', 'б', 'ду', 'л', '▁Ре', 'за', '▁Ш', 'ай', 'х', 'ул', 'ис', 'ла', 'ми', '▁вы', 'раз', 'ил', '▁', 'готов', 'ность', '▁И', 'ран', 'а', '▁о', 'каз', 'ать', '▁со', 'д', 'ей', 'ств', 'ие', '▁', 'прав', 'итель', 'ству', '▁А', 'ф', 'га', 'ни', 'стан', 'а', '▁в', '▁', 'уч', 'ре', 'жд', 'ении', '▁курс', 'ов', '▁про', 'ф', 'ес', 'си', 'он', 'аль', '

In [12]:
# model_rubert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

In [13]:
# инициализируем предобученную модель RuBERT из файла, 
# в json-файле конфигурации описаны параметры модели

# config = ppb.BertConfig.from_json_file('config.json')
# model = ppb.BertModel.from_pretrained('pytorch_model.bin', config = config, from_tf=True)

In [39]:
model_rubert.to('cpu')

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [36]:
import torch

In [45]:
ans = []
for element in list(fin_data['sentence']):
    inputs = tokenizer(element, return_tensors="pt")
    outputs = model_rubert(**inputs)
    ans.append(outputs[0].detach().numpy().argmax()-1)
    #break
    # ans.append(outputs.last_hidden_state)

In [46]:
pd.DataFrame(ans).to_csv('answers24_05.csv', index=False, header=False)

In [15]:
# из-за того, что каждый твит в датасете имеет разную длину (количество токенов)
# мы делаем паддинг - заполнение нулями каждого массива токенов до длины максимального массива
# чтобы на выходе получить матрицу из токенизированных текстов одной длины

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [16]:
# посмотрим на размерность матрицы токенизированных твитов после паддинга

np.array(padded).shape

(6600, 202)

In [17]:
# Накладываем маску на значимые токены
# В данном случае нам важны все слова кроме нулевых токенов, появившихся на предыдущем шаге паддинга

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(6600, 202)

In [ ]:
model_rubert.predict(feature_fin)

In [18]:
# а теперь сформируем вектора текстов с помощью модели RuBERT

# это не быстрый процесс, импортируем инструмент для визуализации времени обработки в цикле
from tqdm import tqdm

# для того, чтобы модель отработала в условиях ограниченных ресурсов - оперативной памяти, мы разделяем входной датасет на батчи.
# при батче в 100 твитов потребление оперативной памяти укладывается в 1Гб
batch_size = 100

# Делаем пустой список для хранения эмбеддингов (векторов) твитов
embeddings = []

for i in tqdm(range(padded.shape[0] // batch_size)):
        # преобразуем батч с токенизированными твитами в тензор 
        # по сути тензор - это многомерный массив, который может быть обработан нейронной сетью
        input_ids = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to('cuda') 
        
        # создаем тензор и для подготовленной маски
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to('cuda')
        
        # передаем в модель BERT тензор из твитов и маску - на выходе получаем эмбеддинги - вектор текста твита
        # torch.no_grad() - для ускорения инференса модели отключим рассчет градиентов
        with torch.no_grad():
               last_hidden_states = model_rubert(input_ids, attention_mask=attention_mask_batch)
        
        # в итоге собираем все эмбеддинги твитов в features
        embeddings.append(last_hidden_states[0][:,0,:].cpu().numpy())



  0%|          | 0/66 [00:01<?, ?it/s]


IndexError: ignored

In [ ]:
# преобразуем список батчей эмбеддингов в numpy-матрицу 
features = np.concatenate(embeddings)

In [ ]:
# выводим размерность полученной матрицы эмбеддингов
# данная модель BERT формирует вектора текстов в 768-мерном пространстве признаков
features.shape

In [ ]:
pip install catboost

In [ ]:
pip install scikit-plot

In [ ]:
# Импортируем необходимые библиотеки для обучения классификатора на logreg и оценки качества

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
import scikitplot as skplt
import seaborn as sns

In [ ]:
# Сохраним целевую переменную: метку тональности позитив/негатив

labels = df_tweets['label']

In [ ]:
labels

In [ ]:
features.shape

In [ ]:
labels.shape

In [ ]:
# Разделяем матрицу признаков и целевую переменную на обучающий и тестовый набор

train_features, test_features, train_labels, test_labels = train_test_split(features, labels[:8000], test_size=0.1, random_state=42)

In [ ]:
catboost_params = {
    'iterations': 10000,
    'learning_rate': 0.001,
    'eval_metric': 'AUC',
    #'eval_metric': 'Accuracy',
    'loss_function': 'MultiClass',
    'task_type': 'GPU',
    'early_stopping_rounds': 1000,
    'use_best_model': True,
    'objective':"MultiClass",
    'verbose': 100
}

In [ ]:
X_tr = pd.DataFrame(train_features) #X_train
X_val =  pd.DataFrame(test_features)
y_tr = train_labels
y_val = test_labels

train_pool = Pool(
    X_tr, 
    y_tr
)
valid_pool = Pool(
    X_val, 
    y_val
)

In [ ]:
model = CatBoostClassifier(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

In [ ]:
import random

In [ ]:
# делаем пробное предсказание
tweet_index = random.randint(1,8000)

print('Text: ' + df_tweets['sentence'][tweet_index])
print('Predict label: ', model.predict(features[tweet_index:tweet_index+1][:])[0])
print('True label: ', df_tweets['label'][tweet_index])


In [ ]:
# оцениваем accuracy на тестовой выборке

model.score(test_features, test_labels)

In [ ]:
d = []
for idx, tt in enumerate(tqdm(df_tweets['sentence'].head(4000))):
    try:
        d.append(
            {
                'text': tt,
                'predict': model.predict(features[idx:idx+1][:])[0][0],
                'gtrue': df_tweets['label'][idx]
            }
        )
    except:
        pass
    #print(max(res,key=itemgetter(1))[0])    
df_train = pd.DataFrame(d)    

In [ ]:
df_train

In [ ]:
df_train.predict.value_counts()

In [ ]:
df_train.gtrue.value_counts()

In [ ]:
import pandas as pd

def confusion_matrix(df: pd.DataFrame, col1: str, col2: str):
    """
    Given a dataframe with at least
    two categorical columns, create a 
    confusion matrix of the count of the columns
    cross-counts
    
    use like:
    
    >>> confusion_matrix(test_df, 'actual_label', 'predicted_label')
    """
    return (
            df
            .groupby([col1, col2])
            .size()
            .unstack(fill_value=0)
            )

In [ ]:
confusion_matrix(df_train,'predict','gtrue')

In [ ]:
# обучаем классификатор на основе логистической регрессии

lr_clf = LogisticRegression(random_state=42, class_weight='balanced')
lr_clf.fit(train_features, train_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=42)

In [ ]:
import random

In [ ]:
# делаем пробное предсказание
tweet_index = random.randint(1,8000)

print('Text: ' + df_tweets['sentence'][tweet_index])
print('Predict label: ', lr_clf.predict(features[tweet_index:tweet_index+1][:])[0])
print('True label: ', df_tweets['label'][tweet_index])


Text: В Венесуэла|Венесуэле 7 октября 2012 года проходят Президентские выборы в Венесуэле (2012)|президентские выборы, которые, по мнению аналитиков, станут самыми напряженными за всю политическую карьеру действующего лидера страны Чавес, Уго|Уго Чавеса, пробывшего у власти почти 14 лет.
Predict label:  0
True label:  0


In [ ]:
# оцениваем accuracy на тестовой выборке

lr_clf.score(test_features, test_labels)

0.5333333333333333

In [ ]:
d = []
for idx, tt in enumerate(tqdm(df_tweets['sentence'].head(4000))):
    try:
        d.append(
            {
                'text': tt,
                'predict': lr_clf.predict(features[idx:idx+1][:])[0],
                'gtrue': df_tweets['label'][idx]
            }
        )
    except:
        pass
    #print(max(res,key=itemgetter(1))[0])    
df_train = pd.DataFrame(d)    
    

100%|██████████| 4000/4000 [00:01<00:00, 2966.93it/s]


In [ ]:
df_train.predict.value_counts()

 0    1876
-1    1138
 1     986
Name: predict, dtype: int64

In [ ]:
df_train.gtrue.value_counts()

 0    2867
-1     616
 1     517
Name: gtrue, dtype: int64

In [ ]:
import pandas as pd

def confusion_matrix(df: pd.DataFrame, col1: str, col2: str):
    """
    Given a dataframe with at least
    two categorical columns, create a 
    confusion matrix of the count of the columns
    cross-counts
    
    use like:
    
    >>> confusion_matrix(test_df, 'actual_label', 'predicted_label')
    """
    return (
            df
            .groupby([col1, col2])
            .size()
            .unstack(fill_value=0)
            )

In [ ]:
confusion_matrix(df_train,'predict','gtrue')

gtrue,-1,0,1
predict,,,
-1,455,627,56
0,91,1692,93
1,70,548,368


In [ ]:
final_data = pd.read_csv('fin_data2.csv', sep='\t')

In [ ]:
final_data.head()

,sentence,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel
0,Абдул Реза Шайхулислами выразил готовность Ира...,Абдул Реза Шайхулислами,PERSON,0,23
1,Абдул Реза Шайхулислами выразил готовность Ира...,афганцев,NATIONALITY,157,165
2,Абдул Реза Шайхулислами выразил готовность Ира...,правительству Афганистана,ORGANIZATION,68,93
3,В своем выступлении Абдул Реза Шайхулислами от...,Абдул Реза Шайхулислами,PERSON,20,43
4,"В свою очередь, Зорар Ахмад Мокбел заявил о то...",Ирана,COUNTRY,86,91


In [ ]:
final_data.shape

(1947, 5)

In [ ]:
final_data.tail()

,sentence,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel
1942,От дальнейших комментариев Брукер воздержалась...,Брукер,PERSON,27,33
1943,"По данным Associated Press, в первый день 2011...",Associated Press,ORGANIZATION,10,26
1944,"По данным Associated Press, в первый день 2011...",Microsoft,ORGANIZATION,62,71
1945,"По данным на конец 2010 года, напоминает AP, у...",AP,ORGANIZATION,41,43
1946,Служба техподдержки Windows Live в ответ на по...,Служба техподдержки Windows Live,ORGANIZATION,0,32


In [ ]:
tokenized_fin = final_data['sentence'].astype(str).str[:512].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)) )
padded_fin = np.array([i + [0]*(max_len-len(i)) for i in tokenized_fin.values])

In [ ]:
attention_mask_f = np.where(padded_fin != 0, 1, 0)
attention_mask_f.shape

(1947, 172)

In [ ]:
# а теперь сформируем вектора текстов с помощью модели RuBERT

# это не быстрый процесс, импортируем инструмент для визуализации времени обработки в цикле
from tqdm import tqdm

# для того, чтобы модель отработала в условиях ограниченных ресурсов - оперативной памяти, мы разделяем входной датасет на батчи.
# при батче в 100 твитов потребление оперативной памяти укладывается в 1Гб
batch_size = 59

# Делаем пустой список для хранения эмбеддингов (векторов) твитов
embeddings_fin = []

for i in tqdm(range(padded_fin.shape[0] // batch_size)):
        # преобразуем батч с токенизированными твитами в тензор 
        # по сути тензор - это многомерный массив, который может быть обработан нейронной сетью
        input_ids_f = torch.LongTensor(padded_fin[batch_size*i:batch_size*(i+1)]).to('cuda') 
        
        # создаем тензор и для подготовленной маски
        attention_mask_batch_f = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to('cuda')
        
        # передаем в модель BERT тензор из твитов и маску - на выходе получаем эмбеддинги - вектор текста твита
        # torch.no_grad() - для ускорения инференса модели отключим рассчет градиентов
        with torch.no_grad():
               last_hidden_states_f = model_rubert(input_ids_f, attention_mask=attention_mask_batch_f)
        
        # в итоге собираем все эмбеддинги твитов в features
        embeddings_fin.append(last_hidden_states_f[0][:,0,:].cpu().numpy())


100%|██████████| 33/33 [00:16<00:00,  2.02it/s]


In [ ]:
features_fin = np.concatenate(embeddings_fin)

In [ ]:
predictions_lr_clf = lr_clf.predict(features_fin)
predictions_model = model.predict(features_fin)

In [ ]:
print(predictions_model)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [ ]:
np.unique(predictions_lr_clf)

array([-1,  0,  1])

In [ ]:
pd.DataFrame(predictions_model).to_csv('catboost_rubert.csv')

In [ ]:
lr_clf_pred = pd.DataFrame(predictions_lr_clf)

In [ ]:
lr_clf_pred.head()

,0
0,-1
1,0
2,-1
3,0
4,0


In [ ]:
lr_clf_pred.rename(columns={0: "ans"})

,ans
0,-1
1,0
2,-1
3,0
4,0
...,...
1942,0
1943,0
1944,-1
1945,0


In [ ]:
lr_clf_pred.to_csv('ans_lr_clf.csv', index=False, header=False)

In [ ]:
new_pred = lr_clf_pred.astype(int)

In [ ]:
new_pred.to_csv('ans.csv', columns=[0])